In [ ]:
import numpy as np
import pandas as pd

In [ ]:
data=pd.read_csv('../data/res_mapping.csv', nrows=5000).dropna()
data['res_dt']=pd.to_datetime(data['res_dt'], format="%d.%m.%y %H:%M")
data['dep_soll']=pd.to_datetime(data['dep_soll'], format="%d.%m.%y %H:%M")
data['diff']=abs((data['res_dt']-data['dep_soll']).dt.total_seconds())
data['dep_soll_date']=data['dep_soll'].astype(str).str[:10]

In [ ]:
tokens = list(set(np.hstack([data['bp_from'], data['bp_to']])))

In [ ]:
def weekday_to_number(day):
    if day == 'Mo':
        return 0
    elif day == 'Di':
        return 1
    elif day == 'Mi':
        return 2
    elif day == 'Do':
        return 3
    elif day == 'Fr':
        return 4
    elif day == 'Sa':
        return 5
    elif day == 'So':
        return 6

In [ ]:
train_mappings=pd.read_csv('../data/train_mapping.csv').dropna()
train_mappings

In [ ]:
for index, row in train_mappings.iterrows():
    vp = row['VP-Tagesleistung (Spaltte R)']
    
    if (len(vp) > 5):
        # Only support one or two day vp
        vp = 'Mo-So'
    days = vp.split('-')
    
    if len(days) == 1:
        train_mappings.at[index, 'VP-Tagesleistung (Spaltte R)'] = [weekday_to_number(days[0])]
    else:
        train_mappings.at[index, 'VP-Tagesleistung (Spaltte R)'] = list(range(weekday_to_number(days[0]), weekday_to_number(days[1])))

In [ ]:
block_mappings=pd.read_csv('../data/block_mapping.csv').dropna()

In [ ]:
for index, row in data.iterrows():
    data.at[index, 'bp_from_token'] = tokens.index(row['bp_from']) / len(tokens)
    data.at[index, 'bp_to_token'] = tokens.index(row['bp_to']) / len(tokens)
    data.at[index, 'res_count'] = len(data.loc[
        np.logical_and(data['train_nr'] == row['train_nr'], 
                       data['dep_soll_date'] == row['dep_soll_date'])])
    weekday = row['dep_soll'].weekday()
    print(weekday)
    block_nr = train_mappings.loc[train_mappings['Zug'] == row['train_nr']].iloc[0, 3]
    found_block = block_mappings.loc[block_mappings['Block Bezeichnung in Jahresformation Fpl-2022'] == block_nr]
    
    if len(found_block) > 0:
        data.at[index, 'max_res_count'] = found_block.iloc[0, 1] 
    else:
        data.at[index, 'max_res_count'] = 0

In [ ]:
data[data['res_count'] > data['max_res_count']]

In [ ]:
counts = data['max_res_count'] - data['res_count']
len(counts.where(counts <= 0).dropna())

In [ ]:
data.to_csv('../data/output.csv', columns = ['diff', 'bp_from_token', 'bp_to_token'], index=False)